In [15]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[1]  # SKN23-2nd-3Team
sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT:", PROJECT_ROOT)

PROJECT_ROOT: /Users/jy/project_2nd/SKN23-2nd-3Team


In [16]:
import json
import torch
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, confusion_matrix
from imblearn.over_sampling import SMOTE
from IPython.display import display, Markdown

from models.model_definitions import MLP_base
from app.utils.metrics import evaluate_churn_metrics
from app.utils.paths import PATHS

In [17]:
print(">>> [MLP_base] Loading Data...")

base_path = PATHS["data_processed"]

anchors = pd.read_parquet(base_path / "anchors.parquet")
features = pd.read_parquet(base_path / "features_ml_clean.parquet")
labels = pd.read_parquet(base_path / "labels.parquet")

for df in [anchors, features, labels]:
    df["user_id"] = df["user_id"].astype(str)

data = anchors.merge(features, on=["user_id", "anchor_time"])
data = data.merge(labels, on=["user_id", "anchor_time"])
data["target"] = (data["label"] == "m2").astype(int)

feature_cols = [c for c in features.columns if c not in ["user_id", "anchor_time"]]
X = data[feature_cols].fillna(0)
y = data["target"].values

>>> [MLP_base] Loading Data...


In [18]:

# 1) split
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, stratify=y, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# 2) SMOTE (train only)
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# 3) scaler (fit on resampled train only)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_res)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# 4) torch tensors (X: float32, y: float32 + (N,1))
Xtr = torch.tensor(X_train_scaled, dtype=torch.float32)
Xva = torch.tensor(X_val_scaled, dtype=torch.float32)
Xte = torch.tensor(X_test_scaled, dtype=torch.float32)

ytr = torch.tensor(
    y_train_res.values if hasattr(y_train_res, "values") else y_train_res,
    dtype=torch.float32
).view(-1, 1)

yva = torch.tensor(
    y_val.values if hasattr(y_val, "values") else y_val,
    dtype=torch.float32
).view(-1, 1)

yte = torch.tensor(
    y_test.values if hasattr(y_test, "values") else y_test,
    dtype=torch.float32
).view(-1, 1)

# 5) loaders
train_loader = DataLoader(TensorDataset(Xtr, ytr), batch_size=256, shuffle=True, drop_last=False)
val_loader   = DataLoader(TensorDataset(Xva, yva), batch_size=256, shuffle=False)
test_loader  = DataLoader(TensorDataset(Xte, yte), batch_size=256, shuffle=False)

In [19]:
MODEL_NAME = "mlp_base"

model = MLP_base(X.shape[1])
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

print(">>> Training start...")
for epoch in range(15):
    model.train()
    total_loss = 0.0

    for xb, yb in train_loader:
        optimizer.zero_grad()

        logits = model(xb)                 # (B, 1) expected
        yb = yb.view(-1, 1).float()         # (B, 1)

        loss = criterion(logits, yb)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1:02d} | Loss: {total_loss/len(train_loader):.4f}")

>>> Training start...
Epoch 01 | Loss: 0.6265
Epoch 02 | Loss: 0.6181
Epoch 03 | Loss: 0.6143
Epoch 04 | Loss: 0.6116
Epoch 05 | Loss: 0.6097
Epoch 06 | Loss: 0.6083
Epoch 07 | Loss: 0.6070
Epoch 08 | Loss: 0.6058
Epoch 09 | Loss: 0.6046
Epoch 10 | Loss: 0.6037
Epoch 11 | Loss: 0.6028
Epoch 12 | Loss: 0.6020
Epoch 13 | Loss: 0.6010
Epoch 14 | Loss: 0.6005
Epoch 15 | Loss: 0.5998


In [20]:
model.eval()
all_targets, all_probs = [], []

with torch.no_grad():
    for xb, yb in test_loader:
        logits = model(xb).view(-1)            # (B,)
        probs = torch.sigmoid(logits).view(-1) # (B,)

        all_targets.extend(yb.view(-1).cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

y_test_np = np.asarray(all_targets, dtype=float)
y_prob_np = np.asarray(all_probs, dtype=float)

metrics = evaluate_churn_metrics(y_test_np, y_prob_np)

In [21]:
summary = {k: v for k, v in metrics.items() if k != "ranking"}
ranking_df = pd.DataFrame(metrics.get("ranking", []))

display(Markdown("### 📊 주요 성능 지표"))
display(pd.DataFrame(summary.items(), columns=["KPI", "Value"]))

display(Markdown("### 📈 Top-K 랭킹"))
display(ranking_df)

### 📊 주요 성능 지표

,KPI,Value
0,PR-AUC (Average Precision),0.89175
1,상위 5% 정밀도 (Precision),0.91788
2,상위 5% 재현율 (Recall),0.05607
3,상위 5% 리프트 (Lift),1.12162


### 📈 Top-K 랭킹

,Top_K,Precision,Recall,Lift
0,5%,0.91788,0.05607,1.12162
1,10%,0.92015,0.11243,1.12440
2,15%,0.91928,0.16849,1.12332
3,20%,0.91604,0.22387,1.11937
4,25%,0.91334,0.27901,1.11606
5,30%,0.91012,0.33363,1.11213


In [22]:
MODEL_ID = "dl__mlp_base"
SPLIT = "test"

EVAL_DIR = PATHS["models_eval"] / "dlmlp_base"
METRICS_DIR = PATHS["models_metrics"]
ASSETS_DIR = PATHS["assets_training"]

for d in [EVAL_DIR, METRICS_DIR, ASSETS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

In [23]:
torch.save(model.state_dict(), PATHS["models_dl"] / f"{MODEL_NAME}.pt")
joblib.dump(scaler, PATHS["models_preprocessing"] / f"{MODEL_NAME}_scaler.pkl")

['/Users/jy/project_2nd/SKN23-2nd-3Team/models/preprocessing/mlp_base_scaler.pkl']

In [24]:
precision, recall, _ = precision_recall_curve(y_test_np, y_prob_np)

fig = plt.figure(figsize=(6,5))
plt.plot(recall, precision)
plt.grid()

plt.close()

In [25]:
def evaluate_and_plot(model, test_loader):
    model.eval()
    all_targets, all_probs = [], []

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs).squeeze()
            probs = torch.sigmoid(outputs)
            all_targets.extend(targets.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    y_true = np.asarray(all_targets)
    y_prob = np.asarray(all_probs)

    metrics = evaluate_churn_metrics(y_true, y_prob)
    ranking_df = pd.DataFrame(metrics.get("ranking", []))

    # PR Curve
    precision, recall, _ = precision_recall_curve(y_true, y_prob)
    pr_auc_val = float(metrics.get("PR-AUC (Average Precision)", 0.0))

    fig_pr = plt.figure(figsize=(6, 5))
    plt.plot(recall, precision, lw=2, label=f"PR-AUC = {pr_auc_val:.4f}")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Precision-Recall Curve")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.close()

    figures = {
        "pr_curve": fig_pr
    }

    return metrics, ranking_df, figures, y_true, y_prob

In [26]:
def save_model_and_artifacts(
    *,
    model,
    model_type,
    model_name,
    model_id,
    split,
    metrics,
    ranking_df,
    y_true,
    y_prob,
    figures,
    scaler=None,
):
    # ---------- paths ----------
    # 저장 경로 정의
    # 저장 경로 정의
    if model_type == "dl":
        EVAL_DIR = PATHS["models_eval"] / f"dl{model_name}"
        MODEL_DIR = PATHS["models_dl"]
    else:
        EVAL_DIR = PATHS["models_eval"] / f"ml{model_name}"
        MODEL_DIR = PATHS["models_ml"]

    METRICS_DIR = PATHS["models_metrics"]
    ASSETS_DIR = PATHS["assets_training"]
    PREP_DIR = PATHS["models_preprocessing"]

    # 디렉토리 생성 (한 번만)
    for d in [EVAL_DIR, METRICS_DIR, ASSETS_DIR, MODEL_DIR, PREP_DIR]:
        d.mkdir(parents=True, exist_ok=True)

    # ---------- model ----------
    torch.save(model.state_dict(), MODEL_DIR / f"{model_name}.pt")

    if scaler is not None:
        import joblib
        joblib.dump(scaler, PREP_DIR / f"{model_name}_scaler.pkl")

    # ---------- model card ----------
    with open(EVAL_DIR / "model_card.json", "w", encoding="utf-8") as f:
        json.dump({
            "model_id": model_id,
            "display_name": model_name,
            "category": model_type.upper(),
            "split": split,
        }, f, indent=2, ensure_ascii=False)

    # ---------- PR metric ----------
    with open(EVAL_DIR / "pr_metrics.json", "w", encoding="utf-8") as f:
        json.dump({
            "model_id": model_id,
            "split": split,
            "pr_auc": float(metrics.get("PR-AUC (Average Precision)", 0.0)),
        }, f, indent=2, ensure_ascii=False)

    # ---------- Top-K ----------
    if not ranking_df.empty:
        base_rate = float(y_true.mean())
        topk_metrics = {
            "model_id": model_id,
            "split": split,
            "base_rate": base_rate,
            "metrics_by_k": [],
        }

        for r in ranking_df.to_dict("records"):
            k = int(str(r["Top_K"]).replace("%", ""))
            topk_metrics["metrics_by_k"].append({
                "k_pct": k,
                "precision_at_k": float(r["Precision"]),
                "recall_at_k": float(r["Recall"]),
                "lift_at_k": float(r["Lift"]),
            })

        with open(EVAL_DIR / "topk_metrics.json", "w", encoding="utf-8") as f:
            json.dump(topk_metrics, f, indent=2, ensure_ascii=False)

    # ---------- cutoffs & confusion ----------
    sorted_scores = np.sort(y_prob)[::-1]
    cutoffs = []

    for k in [5, 10, 15, 30]:
        n = int(len(sorted_scores) * k / 100)
        cutoffs.append({"k_pct": k, "t_k": float(sorted_scores[n-1])})

    with open(EVAL_DIR / "topk_cutoffs.json", "w", encoding="utf-8") as f:
        json.dump({
            "model_id": model_id,
            "split": split,
            "cutoffs_by_k": cutoffs,
        }, f, indent=2, ensure_ascii=False)

    # confusion (Top 5%)
    thr = cutoffs[0]["t_k"]
    y_pred = (y_prob >= thr).astype(int)
    cm = confusion_matrix(y_true, y_pred)

    with open(EVAL_DIR / "confusion_matrix.json", "w", encoding="utf-8") as f:
        json.dump({
            "model_id": model_id,
            "split": split,
            "threshold": thr,
            "matrix": cm.tolist(),
        }, f, indent=2, ensure_ascii=False)

    # ---------- score percentiles ----------
    percentiles = [1, 5, 10, 20, 30, 50]
    scores = np.percentile(y_prob, 100 - np.array(percentiles))

    with open(METRICS_DIR / f"{model_name}_score_percentiles.json", "w", encoding="utf-8") as f:
        json.dump({
            "model_id": model_id,
            "split": split,
            "percentiles": [
                {"pct": p, "score": float(s)}
                for p, s in zip(percentiles, scores)
            ],
        }, f, indent=2, ensure_ascii=False)

 


In [27]:
from pathlib import Path
import sys
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import (
    precision_recall_curve,
    confusion_matrix,
    average_precision_score,
)

PROJECT_ROOT = Path.cwd().parents[1]
sys.path.insert(0, str(PROJECT_ROOT))

from app.utils.save import save_model_and_artifacts

try:
    from app.utils.plotting import configure_matplotlib_korean
    configure_matplotlib_korean()
except Exception:
    pass


def plot_confusion_matrix(
    y_true,
    y_pred,
    title="Confusion Matrix",
    labels=("비이탈(m1)", "이탈(m2)"),
    cmap="Blues",
):
    y_true = np.asarray(y_true).astype(int)
    y_pred = np.asarray(y_pred).astype(int)
    cm = confusion_matrix(y_true, y_pred)

    fig, ax = plt.subplots(figsize=(6, 5))

    im = ax.imshow(cm, cmap=cmap, interpolation="nearest", aspect="equal")
    fig.colorbar(im, ax=ax)

    ax.set_title(title)
    ax.set_xlabel("Predicted (예측값)")
    ax.set_ylabel("Actual (실제값)")

    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)

    thresh = cm.max() / 2.0 if cm.size else 0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(
                j, i, f"{cm[i, j]}",
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black",
                fontsize=12,
            )

    ax.set_xlim(-0.5, cm.shape[1] - 0.5)
    ax.set_ylim(cm.shape[0] - 0.5, -0.5)

    fig.tight_layout()
    return fig


def topk_threshold(y_prob: np.ndarray, k_pct: int) -> float:
    y_prob = np.asarray(y_prob, dtype=float)
    order = np.argsort(-y_prob)
    n_sel = max(int(np.floor(len(y_prob) * k_pct / 100)), 1)
    thr = float(y_prob[order[n_sel - 1]])
    return thr


def plot_confusion_matrix_topk(
    y_true,
    y_prob,
    k_pct: int,
    labels=("비이탈(m1)", "이탈(m2)"),
    cmap="Blues",
):
    y_true = np.asarray(y_true).astype(int)
    y_prob = np.asarray(y_prob).astype(float)

    thr = topk_threshold(y_prob, k_pct)
    y_pred = (y_prob >= thr).astype(int)

    fig = plot_confusion_matrix(
        y_true=y_true,
        y_pred=y_pred,
        title=f"Confusion Matrix (Top {k_pct}%, thr={thr:.5f})",
        labels=labels,
        cmap=cmap,
    )
    return fig


y_true_arr = np.asarray(y_test_np).astype(int)
y_prob_arr = np.asarray(y_prob_np).astype(float)

precision, recall, _ = precision_recall_curve(y_true_arr, y_prob_arr)

pr_auc_val = metrics.get("PR-AUC (Average Precision)")
if pr_auc_val is None:
    pr_auc_val = float(average_precision_score(y_true_arr, y_prob_arr))
else:
    pr_auc_val = float(pr_auc_val)

fig_pr, ax_pr = plt.subplots(figsize=(6, 5))
ax_pr.plot(recall, precision, lw=2, label=f"PR-AUC = {pr_auc_val:.5f}")
ax_pr.set_xlabel("Recall")
ax_pr.set_ylabel("Precision")
ax_pr.set_title("Precision-Recall Curve")
ax_pr.legend()
ax_pr.grid(alpha=0.3)
fig_pr.tight_layout()

k_list = [5, 10, 15, 30]

figures = {
    "pr_curve": fig_pr,
}

for k in k_list:
    figures[f"confusion_matrix_top{k}"] = plot_confusion_matrix_topk(
        y_true_arr,
        y_prob_arr,
        k_pct=k,
        labels=("비이탈(m1)", "이탈(m2)"),
        cmap="Blues",
    )

save_model_and_artifacts(
    model=model,
    model_name="mlp_base",
    model_type="dl",
    model_id="dl__mlp_base",
    split="test",
    metrics=metrics,
    y_true=y_true_arr,
    y_prob=y_prob_arr,
    version="baseline",
    scaler=scaler,
    figures=figures,
)

plt.close(fig_pr)
for k in k_list:
    plt.close(figures[f"confusion_matrix_top{k}"])